# Trabalho de conclusão de curso
## Comparativo de análise de sentimentos em posts do Twitter/Reddit relacionados à Stocks

# Parte 1 - Extração dos Dados

## Preparando ambiente

In [ ]:
# Helpers

!pip install timely --quiet

     |████████████████████████████████| 327kB 5.8MB/s 
     |████████████████████████████████| 266kB 29.0MB/s 


In [ ]:
# Extractors

# !pip install praw --quiet
!pip install psaw --quiet

try: # Gambiarra para instalação do twint que está com problemas e aida não foi resolvido
  import twint
except ImportError:
  errorText = 'RUNTIME WILL STOP.. PLEASE RUN AGAIN!!'
  print(f'\n\x1b[31m{errorText}\x1b[0m\n')

  # !pip3 install twint
  # !pip3 uninstall -y twint
  !pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@master --quiet

  import os
  os.kill(os.getpid(), 9)

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt

from timely import Stopwatch
from datetime import datetime, timedelta

<Figure size 1728x1152 with 0 Axes>

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# def getCredentials(credentialsFile):
#   d = {}
#   with open(credentialsFile) as file:
#     for line in file:

#       (key, val) = line.split(sep='=')
#       d[key.strip().replace(' ', '_').upper()] = val.strip()
#   return d

# credentials = getCredentials('/content/drive/MyDrive/credentials.txt')
# # !cat /content/drive/MyDrive/credentials.txt

In [ ]:
# Constants
SEARCH_STRING = 'TSLA'
SEARCH_LIMIT = 10000
SEARCH_LANG = 'en'

# Dates
SEARCH_DATE = datetime.today().replace(minute = 0, hour = 0, second = 0, microsecond = 0)
SEARCH_DATE_DAYS_BEFORE = timedelta(days = 90)
SEARCH_DATE_SINCE = SEARCH_DATE - SEARCH_DATE_DAYS_BEFORE;

# Other
# REDDIT_SUB = 'wallstreetbets'

## Buscando dados

* Twitter:
  * [Tweepy](https://docs.tweepy.org/en/latest/index.html) *: Twitter API for Python;
  * [Twint](https://github.com/twintproject/twint): Advanced Twitter scraping tool written in Python that allows for scraping Tweets from Twitter profiles without using Twitter's API;
* Reddit:
  * [PRAW](https://praw.readthedocs.io/en/latest/) *: The Python Reddit API Wrapper;
  * [PSAW](https://pypi.org/project/psaw/): A minimalist wrapper for searching public reddit comments/submissions via the pushshift.io API;
  * [BigQuery](https://console.cloud.google.com/bigquery/) *: Dataset fh-bigquery:reddit_comments. (The first 1 TB of query data processed per month is free).

\* Não está sendo utilizado, código comentado.



### Twitter

#### \\\\ Tweepy

In [ ]:
# import tweepy # Tweepy - Python library for accessing the Twitter API.

In [ ]:
# consumer_key = credentials['TWITTER_API_KEY']
# consumer_secret = credentials['TWITTER_API_KEY_SECRET']
# access_token = credentials['TWITTER_ACCESS_TOKEN']
# access_token_secret = credentials['TWITTER_ACCESS_TOKEN_SECRET']

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# tweepyAPI = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
# qtd = SEARCH_LIMIT

# tweets = tweepy.Cursor(
#   tweepyAPI.search,
#   q = SEARCH_STRING,
#   lang = SEARCH_LANG,
#   count = qtd,
#   result_type = 'recent',
#   # until = 'YYYY-MM-DD', # Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
#   include_entities = False,
# ).items(qtd)

In [ ]:
# with Stopwatch() as s:
#   tweetsList = [[tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweets]
#   dfTweepy = pd.DataFrame(tweetsList, columns=['Created At', 'Name', 'Text'])
# print(f'Took {s.duration()}\n')

162.970051


In [ ]:
# dfTweepy.shape

(10000, 3)

In [ ]:
# dfTweepy.head()

,Creaated At,Name,Text
0,2021-03-04 20:09:43,PlutoTradez,Buying $GME dips in my spare time
1,2021-03-04 20:09:39,Mahjestea_,#Stonks will be in the Oxford dictionary and I...
2,2021-03-04 20:09:38,Blaze2408,@mcuban Everyone post and shout out and tag ev...
3,2021-03-04 20:09:36,FrancoisLuciano,And now the power hour starts $GME $AMC
4,2021-03-04 20:09:32,doge2mooon,"I ain’t sorry anymore for that shitty company,..."


In [ ]:
# dfTweepy.tail()

,Creaated At,Name,Text
9995,2021-03-03 21:58:36,petenajarian,RT @MarketRebels: Melvin Capital posts gains o...
9996,2021-03-03 21:58:34,703JV,@pnani456 They are going to make a killing fro...
9997,2021-03-03 21:58:29,MarketRebels,Melvin Capital posts gains of over 22% in Febr...
9998,2021-03-03 21:58:26,DickTugging,"Again ignore our grammar, we are all over hedg..."
9999,2021-03-03 21:58:20,MetatraderRobot,RT @ItaBull: @GRDecter Or a $GME ape


#### Twint

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import twint
tc = twint.Config()

In [ ]:
tc.Search = SEARCH_STRING
# tc.Limit = SEARCH_LIMIT
tc.Lang = SEARCH_LANG
tc.Pandas = True
tc.Hide_output = True
tc.Filter_retweets = True

# tc.Since = SEARCH_DATE_SINCE.strftime('%Y-%m-%d %H:%M:%S')
# tc.Until = SEARCH_DATE.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# # One-time
# with Stopwatch() as s:
#   twint.run.Search(tc)
#   twintTmp = twint.storage.panda.Tweets_df
# print(f'Took {s.duration()}\n')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Took 115.748806



In [ ]:
startDate = datetime(2019, 1, 1)
endDate = datetime(2019, 12, 31)
# endDate = datetime(2019, 1, 30)
deltaDate = timedelta(days = 1)

twintColumns = ['date', 'username', 'tweet']
dfTwint = pd.DataFrame(columns = twintColumns)

In [ ]:
# Loop
with Stopwatch() as s:
  print('Iniciando..')

  while startDate <= endDate:
    tc.Since = startDate.replace(minute = 0, hour = 0, second = 0, microsecond = 0).strftime('%Y-%m-%d %H:%M:%S')
    tc.Until = startDate.replace(minute = 59, hour = 23, second = 59, microsecond = 0).strftime('%Y-%m-%d %H:%M:%S')

    print(f'\n{tc.Since}:')

    twint.run.Search(tc)

    twintTmp = twint.storage.panda.Tweets_df

    if not twintTmp.empty:
      twintTmp = twintTmp[twintColumns]
      dfTwint = dfTwint.append(twintTmp)
    
    startDate += deltaDate
print(f'Took {s.duration()}\n')

In [ ]:
# dfTwint.to_csv('twint_tsla_2019.csv')
# dfTwint = pd.read_csv('twint_tsla_2019.csv', lineterminator = '\n', sep = ',', index_col = 0)

In [ ]:
dfTwint.shape

(530650, 3)

In [ ]:
dfTwint = dfTwint.rename(columns = {'date': 'Created At', 'username': 'Name', 'tweet': 'Text'})
dfTwint.head()

,Created At,Name,Text
0,2019-01-01 23:59:21,alexandrosM,Inspired by thoughts and info I read via @Valu...
1,2019-01-01 23:58:56,Trumpery45,@ShortingIsFun The road stress simulator at 36...
2,2019-01-01 23:58:20,certifiedlink_,@JTSEO9 tsla should have reported a going conc...
3,2019-01-01 23:57:43,smartertrader,Tsla. Very simple trade. If they 352 gets rippy.
4,2019-01-01 23:57:20,ShortingIsFun,@kzdorman @Tesla @elonmusk What are people doi...


In [ ]:
dfTwint['Created At'] = pd.to_datetime(dfTwint['Created At'])
dfTwint.groupby([dfTwint['Created At'].dt.date])['Text'].count()

Created At
2019-01-01     647
2019-01-02    2486
2019-01-03    1593
2019-01-04    1306
2019-01-05     656
              ... 
2019-12-27    1961
2019-12-28     877
2019-12-29     898
2019-12-30    1993
2019-12-31    1603
Name: Text, Length: 365, dtype: int64

In [ ]:
dfTwint.head()

,Created At,Name,Text
0,2019-01-01 23:59:21,alexandrosM,Inspired by thoughts and info I read via @Valu...
1,2019-01-01 23:58:56,Trumpery45,@ShortingIsFun The road stress simulator at 36...
2,2019-01-01 23:58:20,certifiedlink_,@JTSEO9 tsla should have reported a going conc...
3,2019-01-01 23:57:43,smartertrader,Tsla. Very simple trade. If they 352 gets rippy.
4,2019-01-01 23:57:20,ShortingIsFun,@kzdorman @Tesla @elonmusk What are people doi...


### Reddit

#### \\\\ PRAW

In [ ]:
# import praw

In [ ]:
# reddit = praw.Reddit(
#   client_id = credentials['REDDIT_CLIENT_ID'],
#   client_secret = credentials['REDDIT_SECRET'],
#   user_agent = "tcc-bi-master-0.1"
# )

In [ ]:
# with Stopwatch() as s:
#   submissions = reddit.subreddit(REDDIT_SUB).search(
#     SEARCH_STRING,
#     sort = 'new'
#   )
# print(f'Took {s.duration()}\n')

0.000179


In [ ]:
# prawRows = [[submission.created_utc, submission.author, submission.title] for submission in submissions]
# dfPraw = pd.DataFrame(prawRows, columns = ['Created At', 'Name', 'Text'])

# dfPraw.head()

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,Created At,Name,Text
0,1.615052e+09,OwenOnReddit,CONMEBOL have suspended World Cup qualifiers f...
1,1.615026e+09,AdoroQueques,How's Alex Telles?
2,1.614809e+09,gregorcee,Rashford telling Maguire to “shut the f**k up ...
3,1.614803e+09,MatchThreadder,Match Thread: Crystal Palace vs Manchester Uni...
4,1.614605e+09,Arth_,[Loan Watch Report] 22.02-28.02


#### \\\\ PSAW

In [ ]:
# from psaw import PushshiftAPI

# api = PushshiftAPI()

In [ ]:
# gen = api.search_comments(
#   q = SEARCH_STRING,
#   # subreddit = REDDIT_SUB,
#   limit = SEARCH_LIMIT,
#   after = int(SEARCH_DATE_SINCE.timestamp()),
#   before = int(SEARCH_DATE.timestamp())
# )

# max_response_cache = SEARCH_LIMIT
# cache = []

# for c in gen:
#   cache.append(c)
#   if len(cache) >= max_response_cache:
#     break

In [ ]:
# psawRows = [[comment.created_utc, comment.author, comment.body] for comment in cache]
# dfPsaw = pd.DataFrame(psawRows, columns = ['Created At', 'Name', 'Text'])
# dfPsaw['Created At'] = dfPsaw['Created At'].apply(lambda row: datetime.fromtimestamp(row))

# dfPsaw.shape

(1434, 3)

In [ ]:
# dfPsaw.head()

,Created At,Name,Text
0,2021-03-08 23:59:46,bsthedog,"Cathie is a really good investor, but she soun..."
1,2021-03-08 23:59:45,SirrMeowsalot,Waiting for TSLA to get to the 400s\nARKF look...
2,2021-03-08 23:57:44,ANightmareOnBakerSt,Tsla is not worth hundreds of billions of doll...
3,2021-03-08 23:57:37,LordPraetorian,TSLA 1000 EOW
4,2021-03-08 23:57:34,raleighboi,"""Hah old boomer making his last boring trade""\..."


#### BigQuery

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery --project tranquil-petal-234618 dfBigQuery
SELECT f.created_utc, f.author, f.body
FROM `fh-bigquery.reddit_comments.2019_*` as f
WHERE LOWER(f.body) LIKE '%tsla%'

Executing query with job ID: 694788a3-07a3-417a-aae6-9dfc62cb6af2
Query executing: 0.86s


ERROR:
 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/tranquil-petal-234618/queries/694788a3-07a3-417a-aae6-9dfc62cb6af2?maxResults=0&timeoutMs=400&location=US: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/troubleshooting-errors

(job ID: 694788a3-07a3-417a-aae6-9dfc62cb6af2)

          -----Query Job SQL Follows-----          

    |    .    |    .    |    .    |    .    |
   1:SELECT f.created_utc, f.author, f.body
   2:FROM `fh-bigquery.reddit_comments.2018_*` as f
   3:WHERE LOWER(f.body) LIKE '%tsla%'
    |    .    |    .    |    .    |    .    |


In [ ]:
# dfBigQuery.to_csv('bq_tsla_2019.csv')
# dfBigQuery = pd.read_csv('bq_tsla_2019.csv', index_col = 0)

In [ ]:
dfBigQuery.shape

(68907, 3)

In [ ]:
dfBigQuery.rename(columns={'created_utc': 'Created At', 'author': 'Name', 'body': 'Text'}, inplace = True);
dfBigQuery['Created At'] = dfBigQuery['Created At'].apply(lambda row: datetime.fromtimestamp(row))

In [ ]:
dfBigQuery.head()

,Created At,Name,Text
0,2019-07-20 05:46:19,themiamimarlins,Would the Castro fielding error count as a FAR...
1,2019-07-16 00:03:30,BaunDorn,Roll forward means to sell options and repurch...
2,2019-07-21 16:14:27,yaboy_jesse,Makker verklaar mij nog een keer voor duitser ...
3,2019-07-25 02:25:44,BussySlayer69,But TSLA shares aren't backed by a huge milita...
4,2019-07-24 23:08:48,YeOldeFirstTimer,What are everyone’s expectations for TSLA for ...


In [ ]:
dfBigQuery.groupby([dfBigQuery['Created At'].dt.date])['Text'].count()

Created At
2019-01-01     99
2019-01-02    214
2019-01-03    184
2019-01-04    137
2019-01-05     84
             ... 
2019-12-27    246
2019-12-28    171
2019-12-29    139
2019-12-30    298
2019-12-31    252
Name: Text, Length: 365, dtype: int64

### Consolidação dos dados

In [ ]:
dfTwint['Source'] = 'Twitter'
dfBigQuery['Source'] = 'Reddit'

In [ ]:
dfDados = pd.concat([dfTwint, dfBigQuery], ignore_index = True)
dfDados.shape

(599557, 4)

In [ ]:
dfDados['Created At'] = pd.to_datetime(dfDados['Created At'])
dfDados.dtypes

Created At    datetime64[ns]
Name                  object
Text                  object
Source                object
dtype: object

In [ ]:
dfDados

,Created At,Name,Text,Source
0,2019-01-01 23:59:21,alexandrosM,Inspired by thoughts and info I read via @Valu...,Twitter
1,2019-01-01 23:58:56,Trumpery45,@ShortingIsFun The road stress simulator at 36...,Twitter
2,2019-01-01 23:58:20,certifiedlink_,@JTSEO9 tsla should have reported a going conc...,Twitter
3,2019-01-01 23:57:43,smartertrader,Tsla. Very simple trade. If they 352 gets rippy.,Twitter
4,2019-01-01 23:57:20,ShortingIsFun,@kzdorman @Tesla @elonmusk What are people doi...,Twitter
...,...,...,...,...
599552,2019-08-30 21:50:56,AutoModerator,Hey /u/goodlife109! Thank you for your submiss...,Reddit
599553,2019-08-21 07:11:06,AutoModerator,"Hello /u/thethotslayer3000! Unfortunately, you...",Reddit
599554,2019-08-19 14:03:52,AutoModerator,"Hello /u/Thotslay3r69! Unfortunately, your pos...",Reddit
599555,2019-07-08 03:56:56,AutoModerator,"Hello /u/DaveGiantSlayer, this is not really s...",Reddit


In [ ]:
dfDados.to_csv('tsla_2019_data.csv')